# Extract scalar values from JSON

To extract a scalar value from a JSON document, the function [JSON_VALUE](https://docs.microsoft.com/en-us/sql/t-sql/functions/json-value-transact-sql) is available

In [10]:
declare @json varchar(max)
select top (1) @json = json_data from dbo.users_json where id = 1
select json_value(@json, '$.firstName')	

(1 row affected)

Total execution time: 00:00:00.016

(No column name)
John


# Extract non-scalar values from JSON

[JSON_QUERY](https://docs.microsoft.com/en-us/sql/t-sql/functions/json-query-transact-sql) instead allows objects or arrays to be returned

In [11]:
declare @json varchar(max)
select top (1) @json = json_data from dbo.users_json where id = 1
select json_query(@json, '$.address') 


(1 row affected)

Total execution time: 00:00:00.015

(No column name)
"{ ""streetAddress"": ""21 2nd Street"", ""city"": ""New York"", ""state"": ""NY"", ""postalCode"": ""10021-3100"" }"


# Extract Key-Value pairs from JSON

With [OPENJSON](https://docs.microsoft.com/en-us/sql/t-sql/functions/openjson-transact-sql) is possible to operate on JSON in more powerful and flexible ways. The _cross apply_ operator can be thought as a _for each_ operator. Used togheter with `OPENJSON` it enables you to easily operate on all JSON stored into a table column just using a declarative approach. The following code, for example, will extract all key-value pairs from each JSON stored in the `json_data` column of the `dbo.users_json` table.

In [12]:
select
    j.id as document_id,
    t.*
from
    dbo.users_json as j
cross apply
    openjson(j.json_data) as t

(18 rows affected)

Total execution time: 00:00:00.011

document_id,key,value,type
1,firstName,John,1
1,lastName,Smith,1
1,isAlive,true,3
1,age,25,2
1,address,"{ ""streetAddress"": ""21 2nd Street"", ""city"": ""New York"", ""state"": ""NY"", ""postalCode"": ""10021-3100"" }",5
1,phoneNumbers,"[ { ""type"": ""home"", ""number"": ""212 555-1234"" }, { ""type"": ""office"", ""number"": ""646 555-4567"" }, { ""type"": ""mobile"", ""number"": ""123 456-7890"" } ]",4
1,children,[],4
1,spouse,NULL,0
2,firstName,Maggie,1
2,lastName,Doe,1


In case of an array of scalar, for example `{ "children": ["Annette", "Richard"] }`, values will be returned as rows. This ability make really easy to pass arrays to Azure SQL from an external application and use them using regular `SELECT`/`INSERT`/`UPDATE`/`DELETE` commands

In [13]:
select
    c.[value]
from
    dbo.users_json as j
cross apply
    openjson(j.json_data, '$.children') as c

(2 rows affected)

Total execution time: 00:00:00.015

value
Annette
Richard


# Apply Schema-On-Read to JSON

Instead of getting key-value pairs, it is possible to apply a schema to JSON and get a table as result

In [14]:
select
    j.id as document_id,
    t.*
from
    dbo.users_json as j
cross apply
    openjson(j.json_data) with
    (
        FirstName nvarchar(50) '$.firstName',
        LastName nvarchar(50) '$.lastName',
        Age int '$.age',
        [State] nvarchar(50) '$.address.state'
    ) as t

(4 rows affected)

Total execution time: 00:00:00.010

document_id,FirstName,LastName,Age,State
1,John,Smith,25,NY
2,Maggie,Doe,32,NULL
3,Mark,Brown,NULL,NULL
3,Mike,Green,NULL,NULL


If JSON has nested objects, it is possibile to use `OPENJSON` multiple times. In the following sample, `$.phoneNumbers` is passed as a JSON option to a nested `OPENJSON` that operates on it. This time the _outer apply_ is used instead of _cross apply_ as it will return rows even for those JSON documents where `phoneNumbers` element is missing. If you only want to get rows if `phoneNumbers` element exist then you can replace `OUTER APPLY` with `CROSS APPLY`

In [15]:
select
    j.id as document_id,
    t.FirstName,    
    t.LastName,
    t.Age,
    t.State,
    pn.PhoneType,
    pn.PhoneNumber
from
    dbo.users_json as j
cross apply
    openjson(j.json_data) with
    (
        FirstName nvarchar(50) '$.firstName',
        LastName nvarchar(50) '$.lastName',
        Age int '$.age',
        [State] nvarchar(50) '$.address.state',    
        PhoneNumbers nvarchar(max) '$.phoneNumbers' as json
    ) as t
outer apply
    openjson(t.PhoneNumbers) with 
    (
        PhoneType nvarchar(50) '$.type',
        PhoneNumber nvarchar(50) '$.number'
    ) as pn

(8 rows affected)

Total execution time: 00:00:00.010

document_id,FirstName,LastName,Age,State,PhoneType,PhoneNumber
1,John,Smith,25,NY,home,212 555-1234
1,John,Smith,25,NY,office,646 555-4567
1,John,Smith,25,NY,mobile,123 456-7890
2,Maggie,Doe,32,NULL,home,020 7946 0891
2,Maggie,Doe,32,NULL,office,020 7946 0986
2,Maggie,Doe,32,NULL,mobile,123 789-4560
3,Mark,Brown,NULL,NULL,NULL,NULL
3,Mike,Green,NULL,NULL,NULL,NULL
